In [1]:
# 导入所需的库
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import codecs
import unicodedata

import numpy as np
import re

import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\chenmo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
# 读取review文件
df = pd.read_csv('comments_transed6.csv')

cache = pd.read_csv('cache.csv')

In [3]:
df0 = pd.read_csv('comments.csv', encoding='utf-8')

In [4]:
len(df0['listing_id'].unique())

6209

In [5]:
cache.shape

(597, 2)

In [6]:
df = df.rename(columns={'comments': 'comments_trans'})
new_df = pd.concat([df0, df['comments_trans']], axis=1)
new_df.columns.tolist()

['listing_id',
 'id',
 'date',
 'reviewer_id',
 'reviewer_name',
 'comments',
 'comments_trans']

In [90]:
null_matrix = new_df['comments_trans'].isnull()

len(np.where(null_matrix == True)[0])

0

In [8]:
new_df = new_df.dropna()

In [9]:
new_df.to_csv('comment_cleaned1.csv', index=False)

## 转换

In [35]:
# 读取review文件
reviews_df = pd.read_csv('comment_cleaned1.csv')

In [11]:
tmp = reviews_df['comments_trans']

# remove stop words
stopwords = set(nltk.corpus.stopwords.words('english'))
tmp = tmp.apply(lambda x: ' '.join([word for word in x.split() if word not in stopwords]))

# Segment the comments column
reviews_df['words'] = tmp.apply(lambda x: nltk.word_tokenize(x))

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(tmp)

# Instantiate a CountVectorizer
vec = CountVectorizer()

In [12]:
test_i = reviews_df['words'][76417]
print(test_i)
vec.fit_transform(test_i)

['�', '?', '�', '?', '�', '?', '�', '?', '�', '?', '�', '?', '�', '?', '�', '?', '�', '?', '�', '?', '�', '?', '�', '?', '�', '?', '�', '?', '�', '?', '�', '?', '�', '?', '�', '?', '�', '?', '�', '?']


ValueError: empty vocabulary; perhaps the documents only contain stop words

In [ ]:
reviews_df['words'][2956] = ['Image']

reviews_df['words'][17800] = ['Normal']

reviews_df['words'][20266] = ['prefect']

reviews_df['words'][24477] = ['normal']

reviews_df['words'][30336] = ['bad']

reviews_df['words'][32539] = ['normal']


In [ ]:
i = reviews_df['listing_id']
i

In [ ]:
it = ['C']

print(len(it) == 1)

if len(it) == 1 and len(it[0]) == 1 or len(it) == 0:
    print('111')

In [ ]:
# 用词袋模型将分词后的文本转换成文本向量
# reviews_df['vector'] = reviews_df['words'].apply(lambda x: vec.fit_transform(x).toarray())

temp = []
for i in range(len(reviews_df['words'])):
    item = reviews_df['words'][i]

    print(i, item)

    if len(item) == 1 and len(item[0]) == 1 or len(item) == 0:
        reviews_df['words'][i] = ['normal']

    res = vec.fit_transform(reviews_df['words'][i]).toarray()
    temp.append(res)

In [ ]:
reviews_df['vector'] = temp

In [ ]:
reviews_df.to_csv('comment_cleaned_vec.csv', index=False)

In [ ]:
len(['D222'][0])

In [16]:
# 根据listing_id计算文本向量的平均值
# data_mean = reviews_df.groupby('listing_id')['vector'].mean()

df_mean = pd.DataFrame(columns=['listing_id','data_mean'])
listing_ids = reviews_df['listing_id'].unique()


In [17]:
for listing_id in listing_ids:
    vectors = reviews_df[reviews_df['listing_id']==listing_id]['vector']

    list_means = []
    for vector in vectors:
        list_means.append(np.mean(vector))

    #计算整个列表的均值
    final_mean = np.mean(list_means)
    
    df_mean = df_mean.append({'listing_id':listing_id,'data_mean':final_mean}, ignore_index=True)

df_mean

TypeError: cannot perform reduce with flexible type

In [ ]:
#将每个listing_id的vector均值添加到原来的df中
reviews_df = reviews_df.merge(df_mean, on='listing_id')

In [ ]:
reviews_df.columns.tolist()

In [ ]:
reviews_df.to_csv('comment_cleaned_vec.csv', index=False)

## 合并listing

In [73]:
# 读取listing文件
listing_df = pd.read_csv('cleaned_data.csv')
reviews_df = pd.read_csv('comment_cleaned_vec.csv')

In [74]:
listing_df.shape

(7566, 45)

In [75]:
reviews_df.shape

(234365, 10)

In [76]:
reviews_df.columns.tolist()

['listing_id',
 'id',
 'date',
 'reviewer_id',
 'reviewer_name',
 'comments',
 'comments_trans',
 'words',
 'vector',
 'data_mean']

In [77]:
data_mean_df = reviews_df[['listing_id', 'data_mean']].copy()
data_mean_df.shape

(234365, 2)

In [78]:
data_mean_df.rename(columns={'listing_id': 'id'}, inplace=True)
data_mean_df.rename(columns={'data_mean': 'review_mean'}, inplace=True)

In [79]:
len(data_mean_df['id'].unique())

5185

In [82]:
data_mean_df.head()

,id,review_mean
0,44077,0.061733
256,302634,0.067611
360,1797712,0.039422
427,374792,0.064033
462,386860,0.059420


In [81]:
data_mean_df.drop_duplicates(subset=None, keep='first', inplace=True)

In [83]:
listing_df = listing_df.merge(data_mean_df, on='id')

In [84]:
listing_df.head()

,Unnamed: 0,id,host_since,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_verifications,host_has_profile_pic,host_identity_verified,...,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,host_days_active,verification_count,property_type_category,bathrooms_type,amenities_count,review_mean
0,0,44077,2010-08-06,4,1,3,1,1,1,1,...,0.0,2.0,0.0,1.85,4419.0,2,2,6,994,0.061733
1,1,85156,2010-08-06,4,1,3,1,1,1,1,...,0.0,2.0,0.0,1.53,4419.0,2,2,6,657,0.066811
2,2,159889,2011-07-01,4,1,3,0,1,1,1,...,0.0,3.0,0.0,2.78,4090.0,2,3,8,491,0.086331
3,3,162809,2011-07-04,4,3,2,1,1,1,1,...,0.0,2.0,0.0,3.68,4087.0,2,3,6,892,0.088974
4,4,165828,2011-07-07,4,1,3,0,1,1,1,...,5.0,0.0,0.0,0.48,4084.0,2,0,3,482,0.079291


In [71]:
listing_df.shape

(5185, 46)

In [86]:
listing_df.drop('Unnamed: 0', axis=1, inplace=True)

In [87]:
listing_df.columns.tolist()

['id',
 'host_since',
 'host_response_time',
 'host_response_rate',
 'host_acceptance_rate',
 'host_is_superhost',
 'host_verifications',
 'host_has_profile_pic',
 'host_identity_verified',
 'neighbourhood_cleansed',
 'latitude',
 'longitude',
 'property_type',
 'room_type',
 'accommodates',
 'bathrooms_text',
 'bedrooms',
 'beds',
 'amenities',
 'price',
 'minimum_nights',
 'maximum_nights',
 'number_of_reviews',
 'number_of_reviews_ltm',
 'number_of_reviews_l30d',
 'first_review',
 'review_scores_rating',
 'review_scores_accuracy',
 'review_scores_cleanliness',
 'review_scores_checkin',
 'review_scores_communication',
 'review_scores_location',
 'review_scores_value',
 'instant_bookable',
 'calculated_host_listings_count',
 'calculated_host_listings_count_entire_homes',
 'calculated_host_listings_count_private_rooms',
 'calculated_host_listings_count_shared_rooms',
 'reviews_per_month',
 'host_days_active',
 'verification_count',
 'property_type_category',
 'bathrooms_type',
 'amenit

In [88]:
listing_df['review_mean'].isnull().any()

False

In [89]:
listing_df.to_csv('cleaned_data_review.csv', index=False)